In [54]:
import random
import csv
import gensim
import ast
import pickle
from numpy import array, mean
from gensim.test.utils import datapath
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from gensim import corpora, models
#import sklearn.externals.joblib as extjoblib
import joblib
from sklearn import svm
#from sklearn.externals import joblib


def createTrainingDataset():
    GISWDataLength = 1700
    GIDataLength = 3000
    count = 0
    corpus = []
    print("Extracting training data: ")
    with open('cleanedGeneralIssuesCommonTS1.csv', mode='r') as commonReader:
        commonReader.readline()
        csvGISWReader = csv.reader(commonReader, delimiter=',')
        for row in csvGISWReader:
            string = row[5]
            string = [str(x) for x in string.strip().split()]
            if len(string) <= 1:
                continue
            corpus.append([1, row[1], string])
            count += 1
            if count > GISWDataLength:
                break
    for i in corpus[0:5]:
        print(i)

    count = 0
    with open('cleanedGeneralIssuesTS1.csv', mode='r') as generalReader:
        generalReader.readline()
        csvGIReader = csv.reader(generalReader, delimiter=',')
        for row in csvGIReader:
            string = row[5]
            string = [str(x) for x in string.strip().split()]
            if len(string) <= 1 or row[1] == "[deleted]":
                continue
            corpus.append([0, row[1], string])
            count += 1
            if count > GIDataLength:
                break
    for i in corpus[-5:]:
        print(i)

    random.shuffle(corpus)
    print("SHUFFLING->>>>>>>>>>>>>")
    for i in corpus[0:5]:
        print(i)
    for i in corpus[-5:]:
        print(i)

    with open('training1-copy.csv', mode='w') as writer:
        csvWriter = csv.writer(writer, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvWriter.writerow(['Class', 'Author', 'Text'])
        for row in corpus:
            csvWriter.writerow(row)
    print("Done Extracting")
    pass




In [55]:
def createTestingDataset():
    GISWDataLength = 680
    GIDataLength = 3000
    count = 0
    corpus = []
    print("Extracting testing data: ")
    with open('cleanedGeneralIssuesCommonTS2.csv', mode='r') as commonReader:
        commonReader.readline()
        csvGISWReader = csv.reader(commonReader, delimiter=',')
        for row in csvGISWReader:
            string = row[5]
            string = [str(x) for x in string.strip().split()]
            if len(string) <= 1:
                continue

            count += 1
            #if (count < 240):
            #    continue
            corpus.append([1, row[1], string])
            if count > GISWDataLength:
                break
    for i in corpus[0:5]:
        print(i)

    count = 0
    with open('cleanedGeneralIssuesTS2.csv', mode='r') as generalReader:
        generalReader.readline()
        csvGIReader = csv.reader(generalReader, delimiter=',')
        for row in csvGIReader:
            string = row[5]
            string = [str(x) for x in string.strip().split()]
            if len(string) <= 1 or row[1] == "[deleted]":
                continue
            count += 1
            #if (count < 5000):
            #   continue
            corpus.append([0, row[1], string])
            if count > GIDataLength :
                break
    for i in corpus[-5:]:
        print(i)

    random.shuffle(corpus)
    print("SHUFFLING->>>>>>>>>>>>>")
    for i in corpus[0:5]:
        print(i)
    for i in corpus[-5:]:
        print(i)

    with open('testing1.csv', mode='w') as writer:
        csvWriter = csv.writer(writer, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvWriter.writerow(['Class', 'Author', 'Text'])
        for row in corpus:
            csvWriter.writerow(row)
    print("Done Extracting")
    pass

In [56]:
def trainingModel():
    corpus = []
    docNumber = 0
    with open('training1-copy.csv', mode='r', encoding="utf-8") as reader:
        reader.readline()
        csvReader = csv.reader(reader)
        for row in csvReader:
            if len(row)>=3:
                corpus.append([str(x) for x in row[2].strip().split()])
                docNumber += 1

    print("Total Count: ", docNumber)
    dictionary = gensim.corpora.Dictionary(corpus)
    count = 0
    for k, v in dictionary.iteritems():
        print(k, v)
        count += 1
        if count > 10:
            break
    print(len(dictionary))
    dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=450)
    print(len(dictionary))
    #count = 0
    #for k, v in dictionary.iteritems():
     #   print(k, v)
      #  count += 1
       # if count > 10:
        #    break
    #print("corpus:")
    #print(corpus[1:100])
    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    bow_corpus[1500]
    #print("bow_corpus:")
    #print(bow_corpus[1:100])
    
    bow_doc_4310=bow_corpus[1500]
    for i in range(len(bow_doc_4310)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],
                                                         dictionary[bow_doc_4310[i][0]],
                                                         bow_doc_4310[i][1]))
    
    """tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    from pprint import pprint
    for doc in corpus_tfidf:
        pprint(doc)
        break
    """
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=2, id2word=dictionary, passes=10, workers=4)
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
        
        
    corpus = []
    docNumber = 0
    with open('testing1-copy.csv', mode='r', encoding="utf-8") as reader:
        reader.readline()
        csvReader = csv.reader(reader)
        for row in csvReader:
            if len(row)>=3:
                corpus.append([str(x) for x in row[2].strip().split()])
                docNumber += 1

    print("Total Count: ", docNumber)
    dictionary = gensim.corpora.Dictionary(corpus)
    count = 0
    for k, v in dictionary.iteritems():
        print(k, v)
        count += 1
        if count > 10:
            break
    print(len(dictionary))
    dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=450)
    print(len(dictionary))
    
    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    #bow_corpus[5912]
    #print("bow_corpus:")
    #print(bow_corpus[1:100])
    
    bow_doc_4310=bow_corpus[770]
    for i in range(len(bow_doc_4310)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],
                                                         dictionary[bow_doc_4310[i][0]],
                                                         bow_doc_4310[i][1]))
    
   
    print(lda_model.get_document_topics(bow_doc_4310)) 
    """    
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=2, id2word=dictionary, passes=1, workers=4)
    
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
    corpus[5912]
    for index, score in sorted(lda_model[bow_corpus[5912]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    
    for index, score in sorted(lda_model_tfidf[bow_corpus[5912]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    # save the model to disk
    #modelFile = datapath('/Users/Dell/final major project/ldaTrainedModel')
    #lda_model.save(modelFile)
    #filename = 'finalized_model.sav'
    #pickle.dump(model, open(filename, 'wb'))"""
    pass


In [57]:
def bestTopicModel():
    searchParams = {'n_components': [n for n in range(2, 7)], 'learning_decay': [0.3,0.5,0.7,0.9]}
    lda = LatentDirichletAllocation()
    model = GridSearchCV(lda, searchParams)

    corpus = []
    docNumber = 0
    with open('training1-copy.csv', mode='r') as reader:
        reader.readline()
        csvReader = csv.reader(reader)
        for row in csvReader:
            if len(row)>=1:
                if (int(row[0]) == 1):
                    corpus.append(" ".join(ast.literal_eval(row[2])))
                    docNumber += 1
    print("Total Doc: ", docNumber)
    for i in corpus[0:5]:
        print(i)

    vectorizer = CountVectorizer(min_df=10)
    data_vectorized = vectorizer.fit_transform(corpus)
    print('Started Training')
    model.fit(data_vectorized)
    print('Found the best model')
    # Best Model
    best_lda_model = model.best_estimator_

    # Model Parameters
    print("Best Model's Params: ", model.best_params_)

    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)

    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
    filename = 'finalized_LDAModel.sav'
    pickle.dump(best_lda_model, open(filename, 'wb'))

    pass

In [58]:
def getTrainFeatures(filename):
    corpus = []
    docNumber = 0
    suicidalCount = 0
    nonsuicidalCount = 0
    with open(filename, mode='r') as reader:
        reader.readline()
        csvReader = csv.reader(reader)
        for row in csvReader:
            if len(row)>=1:
                corpus.append(array([row[0], " ".join(ast.literal_eval(row[2]))]))
                if (row[0] == '1'):
                    suicidalCount += 1
                else:
                    nonsuicidalCount += 1
                docNumber += 1
    print("Total Doc: ", docNumber)
    print("Suicidal Count: ", suicidalCount)
    print("Non-Suicidal Count: ", nonsuicidalCount)
    corpus = array(corpus)
    print(corpus[:,1])

    vectorizer = CountVectorizer(min_df=10)
    data_vectorized = vectorizer.fit_transform(corpus[:, 1])
    print(data_vectorized.shape)
    print(vectorizer.vocabulary_.get('depress'))

    tf_transformer = TfidfTransformer(use_idf=False).fit(data_vectorized)
    X_train_tf = tf_transformer.transform(data_vectorized)
    print(X_train_tf.shape)

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(data_vectorized)
    print(X_train_tfidf.shape)

    return X_train_tfidf, corpus[:, 0], vectorizer, tfidf_transformer



In [59]:
def getTestFeatures(filename):
    corpus = []
    docNumber = 0
    suicidalCount = 0
    nonsuicidalCount = 0
    with open(filename, mode='r') as reader:
        reader.readline()
        csvReader = csv.reader(reader)
        for row in csvReader:
            if len(row)>=1:
                corpus.append(array([row[0], " ".join(ast.literal_eval(row[2]))]))
                if(row[0] == '1'):
                    suicidalCount += 1
                else:
                    nonsuicidalCount += 1
                docNumber += 1
    print("Total Doc: ", docNumber)
    print("Suicidal Count: ", suicidalCount)
    print("Non-Suicidal Count: ", nonsuicidalCount)
    corpus = array(corpus)
    return corpus
    # print(corpus[:,1])

    # vectorizer = CountVectorizer(min_df=10)
    # data_vectorized = vectorizer.transform(corpus[:, 1])
    # print(data_vectorized.shape)
    # print(vectorizer.vocabulary_.get('depress'))
    #
    # tf_transformer = TfidfTransformer(use_idf=False).fit(data_vectorized)
    # X_train_tf = tf_transformer.transform(data_vectorized)
    # print(X_train_tf.shape)
    #
    # tfidf_transformer = TfidfTransformer()
    # X_train_tfidf = tfidf_transformer.fit_transform(data_vectorized)
    # print(X_train_tfidf.shape)

    # return X_train_tfidf, corpus[:, 0]



In [72]:
def SVM():
    X_train_tfidf, target, countVectorizer, tfidf_transformer = getTrainFeatures('training1-copy.csv')
    print('Started Training...')
    #clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=500, tol=None)
    #clf = MultinomialNB()
    #clf = svm.SVC(kernel='rbf')
    clf = RandomForestClassifier(max_depth = 6, random_state=0)
    clf.fit(X_train_tfidf, target)
    print('Done Training...')
    print('Started Testing...')

    testingCorpus = getTestFeatures('testing1.csv')
    X_test_counts = countVectorizer.transform(testingCorpus[:, 1])
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    predictions = clf.predict(X_test_tfidf)


    print(len(target))
    predictions = clf.predict(X_test_tfidf)
    print(predictions)
    print(target)
    print(mean(predictions == testingCorpus[:, 0]))
    print(confusion_matrix(testingCorpus[:, 0], predictions))
    print(confusion_matrix(testingCorpus[:, 0], predictions).ravel())

    """validatingCorpus = getTestFeatures('validation1.csv')
    X_test_counts = countVectorizer.transform(validatingCorpus[:, 1])
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    predictions = clf.predict(X_test_tfidf)

    # print(len(test_target))
    # predictions = clf.predict(X_test_tfidf)
    print(predictions)
    # print(test_target)
    print(mean(predictions == validatingCorpus[:, 0]))
    print(confusion_matrix(validatingCorpus[:, 0], predictions))
    print(confusion_matrix(validatingCorpus[:, 0], predictions).ravel())
"""


In [61]:
def main():
    createTrainingDataset()
    #createTestingDataset()
    #createValidationDataset()
    #trainingModel()
    #bestTopicModel()
    # SVM()
    pass


if __name__ == '__main__':
    main()

Extracting training data: 
[1, 'Plassholdertekst', ['feel', 'like', 'actual', 'feel', 'normal', 'happi', 'fall', 'hard', 'happi', 'alway', 'part', 'cycl', 'depress']]
[1, 'Limnir', ['constantli', 'wish', 'peopl', 'dead', 'cancer', 'recent', 'older', 'stepbroth', 'live', 'sick', 'constantli', 'tell', 'younger', 'brother', 'wish', 'dead', 'never', 'second', 'neither', 'say', 'like', 'around', 'time', 'sick', 'wish', 'peopl', 'dead', 'time', 'say', 'face', 'someth', 'like', 'work', 'one', 'thing', 'time', 'everyon', 'els', 'littl', 'brother', 'time', 'say', 'turn', 'peopl', 'anyth', 'littl', 'fix', 'broken', 'time', 'everyth', 'tell', 'wish', 'kill', 'older', 'stepbroth', 'say', 'pay', 'hous', 'noth', 'brother', 'think', 'sick', 'think', 'normal', 'wish', 'littl', 'brother', 'dead']]
[1, 'Plassholdertekst', ['often', 'final', 'feel', 'realli', 'happi', 'good', 'suddenli', 'realli', 'bad', 'could', 'mean', 'bipolar', 'type', 'happi', 'real']]
[1, 'SHThrowAway213', ['let', 'know', 'said', '

Done Extracting


In [54]:
trainingModel()

Total Count:  3352
0 'caus',
1 'cover',
2 'everyth']
3 'get',
4 'go',
5 'help',
6 'hospit',
7 'insur',
8 'know',
9 'never',
10 'pay',
5697
450
Word 1 ("'get',") appears 1 time.
Topic: 0 
Words: 0.018*"'time'," + 0.015*"'know'," + 0.014*"'one'," + 0.013*"'even'," + 0.013*"'would'," + 0.013*"'life'," + 0.013*"'realli'," + 0.012*"'tri'," + 0.012*"'peopl'," + 0.009*"'think',"
Topic: 1 
Words: 0.028*"'get'," + 0.025*"'want'," + 0.018*"'know'," + 0.016*"'depress'," + 0.014*"'peopl'," + 0.013*"'time'," + 0.013*"'think'," + 0.012*"'life'," + 0.011*"'day'," + 0.011*"'work',"
Total Count:  4108
0 'dose',
1 'littl']
2 'look',
3 'much',
4 'see',
5 'time',
6 ['first',
7 'affect',
8 'almost',
9 'alon',
10 'along',
5920
450
Word 0 ("'look',") appears 1 time.
Word 1 ("'much',") appears 3 time.
Word 2 ("'see',") appears 2 time.
Word 3 ("'time',") appears 3 time.
Word 4 ("'almost',") appears 1 time.
Word 5 ("'alon',") appears 1 time.
Word 9 ("'basic',") appears 1 time.
Word 10 ("'broke',") appears 1 tim

In [32]:
bestTopicModel()

Total Doc:  997
two usual depress therapi quit expens though wonder anyon provid good insight like know therapist like gener idea also use impact peopl wonder treatment right thank
assum ask hate probabl known mani
well put feel given life without choic never truli meant born world drag age young even think sort way life preciou give still young true genuin mental emot somewhat physic disabl old mother left anyon lean ye step father half sister mention rest famili give support take much intern pain given govern money known social secur disabl incom young naiv sat told realli cours abl help social secur took told lack date back sent profession help group peopl similar sent mani time age old old time due sent place rest alway suicid patient still way younger major depress high anxieti panic autist spectrum disord function limb weak suicid black cognit suffer cancer babi gone chemotherapi make huge toll affect life drastic graduat high school diploma certif complet get colleg tri go ged s

In [65]:
#SVM()

In [15]:
#trainingModel()

In [67]:
#sgd
SVM()

Total Doc:  4552
Suicidal Count:  1551
Non-Suicidal Count:  3001
['nobodi left someon els worst part suffer self day feel time feel empti feel stomach feel physic sensat hopeless brain battl want die hand know noth go ever chang tri act impuls everyth plain hard feel rage act public hard break cri walk tri show normal fall love someon hurt replac willingli chose block commun person talk someon els leav broke suicid knew like said want die choic point'
 'hello look book read one self help get depress one enough get mind neg depress make sens thank'
 'bad immatur old back huge free spirit know immatur age howev sinc 2012 come back bite as two side know hard young stupid time also feel like deserv feel like anyth illeg know hurt lot peopl two year period old shell former teen self big heart love gentl would anyth anyon rude immatur selfish person want eventu help peopl especi want apolog everyon hurt also want open old wound know want feel better move life time think happi move life would

In [69]:
#multinomialNB
SVM()

Total Doc:  4552
Suicidal Count:  1551
Non-Suicidal Count:  3001
['nobodi left someon els worst part suffer self day feel time feel empti feel stomach feel physic sensat hopeless brain battl want die hand know noth go ever chang tri act impuls everyth plain hard feel rage act public hard break cri walk tri show normal fall love someon hurt replac willingli chose block commun person talk someon els leav broke suicid knew like said want die choic point'
 'hello look book read one self help get depress one enough get mind neg depress make sens thank'
 'bad immatur old back huge free spirit know immatur age howev sinc 2012 come back bite as two side know hard young stupid time also feel like deserv feel like anyth illeg know hurt lot peopl two year period old shell former teen self big heart love gentl would anyth anyon rude immatur selfish person want eventu help peopl especi want apolog everyon hurt also want open old wound know want feel better move life time think happi move life would

In [71]:
#SVM
SVM()

Total Doc:  4552
Suicidal Count:  1551
Non-Suicidal Count:  3001
['nobodi left someon els worst part suffer self day feel time feel empti feel stomach feel physic sensat hopeless brain battl want die hand know noth go ever chang tri act impuls everyth plain hard feel rage act public hard break cri walk tri show normal fall love someon hurt replac willingli chose block commun person talk someon els leav broke suicid knew like said want die choic point'
 'hello look book read one self help get depress one enough get mind neg depress make sens thank'
 'bad immatur old back huge free spirit know immatur age howev sinc 2012 come back bite as two side know hard young stupid time also feel like deserv feel like anyth illeg know hurt lot peopl two year period old shell former teen self big heart love gentl would anyth anyon rude immatur selfish person want eventu help peopl especi want apolog everyon hurt also want open old wound know want feel better move life time think happi move life would

In [73]:
#random forest
SVM()

Total Doc:  4552
Suicidal Count:  1551
Non-Suicidal Count:  3001
['nobodi left someon els worst part suffer self day feel time feel empti feel stomach feel physic sensat hopeless brain battl want die hand know noth go ever chang tri act impuls everyth plain hard feel rage act public hard break cri walk tri show normal fall love someon hurt replac willingli chose block commun person talk someon els leav broke suicid knew like said want die choic point'
 'hello look book read one self help get depress one enough get mind neg depress make sens thank'
 'bad immatur old back huge free spirit know immatur age howev sinc 2012 come back bite as two side know hard young stupid time also feel like deserv feel like anyth illeg know hurt lot peopl two year period old shell former teen self big heart love gentl would anyth anyon rude immatur selfish person want eventu help peopl especi want apolog everyon hurt also want open old wound know want feel better move life time think happi move life would